In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

In [3]:
# POINT TO TARGET SITE 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
# PULL DATA FROM SITE

html_doc = requests.get(url).text
#html_doc

In [5]:
# CREATE BEAUTIFUL SOUP OBJECT 

from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

#print(soup.prettify())

In [6]:
# INITIALIZE 4 LISTS TO COLLECT DATA

a0 = []
a1 = []
a2 = []
a3 = []

In [7]:
# PARSE DATA AND STORE IN LISTS 

n = 0
for string in soup.table.stripped_strings:

    if(n==3):
        #print("n=3")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a3.append(entry)
        n=0
    
    if(n==2):
        #print("n=2")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a2.append(entry)
        n=3

    if(n==1):
        #print("n=1")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a1.append(entry)
        n=2  

    if(n==0):
        #print("n=0")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a0.append(entry)
        n=1

In [8]:
# EXTRACT RELEVANT LISTS AND PUT INTO DF
df0 = pd.DataFrame(a0)
df1 = pd.DataFrame(a1)
df2 = pd.DataFrame(a2)

In [9]:
df_final = pd.concat([df0, df1, df2], axis=1)

In [10]:
df_final.head()

,0,0,0
0,'Postcode','Borough','Neighbourhood'
1,'M1A','Not assigned','Not assigned'
2,'M2A','Not assigned','Not assigned'
3,'M3A','North York','Parkwoods'
4,'M4A','North York','Victoria Village'


In [11]:
# MAKE 1ST ROW THE HEADER 

df_final.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_final = df_final[1:]

In [13]:
# REMOVE SPECIAL CHARACTERS 

df_final["Borough"] = df_final["Borough"].str.replace("\'", '')
df_final["Postcode"] = df_final["Postcode"].str.replace("\'", '')
df_final["Neighbourhood"] = df_final["Neighbourhood"].str.replace("\'", '')

In [14]:
df_final.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [15]:
# REMOVE ALL BOROUGHS WHICH HAVE "NOT ASSIGNED"
df_final2 = df_final.query('Borough != "Not assigned"').copy()
df_final2.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [16]:
# REPLACE ANY NOT ASSIGNED OCCURENCES IN NEIGHBOURHOOD WITH BOROUGH NAME
for index, row in df_final2.iterrows():
    if row['Neighbourhood'] == "Not assigned":
        row['Neighbourhood'] = row['Borough']

In [17]:
# CHECK
df_final2.query('Borough == "Not assigned"')

,Postcode,Borough,Neighbourhood


In [18]:
# COMBINE NEIGHBOURHOOD IF POSTCODE SAME
df_final3 = df_final2.groupby(['Postcode','Borough'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))

In [19]:
df_final3.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
df_final3.shape

(103, 3)